# **2026 NFL Big Data Bowl**
Anthony Thuesday

1.) Libraries

In [3]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from pathlib import Path
import random
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
import statsmodels.api as sm
import catboost as cb
from catboost import CatBoostRegressor



2.) Getting Data Ready

In [6]:

def load_and_merge_nfl_data(input_path, output_path):

    #Load
    input_files = sorted(glob.glob("nfl-big-data-bowl-2026-prediction/train/input_2023_w*.csv"))
    output_files = sorted(glob.glob("nfl-big-data-bowl-2026-prediction/train/output_2023_w*.csv"))

    if not input_files or not output_files:
        raise FileNotFoundError("No input or output files found. Check paths or patterns.")

    aggra_input = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
    aggra_output = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)

    print(f"Loaded {len(input_files)} input files ({aggra_input.shape[0]:,} rows)")
    print(f"Loaded {len(output_files)} output files ({aggra_output.shape[0]:,} rows)")


    aggra_output = aggra_output.rename(columns={"x": "x_output", "y": "y_output"})
    merge_keys = ["game_id", "play_id", "nfl_id", "frame_id"]
    df = aggra_input.merge(
        aggra_output,
        on=merge_keys,
        how="left"
    )

    #Validation checks
    print("\n✅ Merge summary:")
    print(f"  Input shape: {aggra_input.shape}")
    print(f"  Output shape: {aggra_output.shape}")
    print(f"  Merged shape: {df.shape}")


    na_check_x = df.loc[df["x_output"].isna(), "player_to_predict"].eq(False).all()
    na_check_y = df.loc[df["y_output"].isna(), "player_to_predict"].eq(False).all()



    print("\nCrosstab (who has NaN outputs):")
    print(pd.crosstab(df["player_to_predict"], df["x_output"].isna(),
                      rownames=["player_to_predict"], colnames=["x_output_is_NaN"]))

    return df


In [7]:
df = load_and_merge_nfl_data(
    "nfl-big-data-bowl-2026-prediction/train/input_2023_w*.csv",
    "nfl-big-data-bowl-2026-prediction/train/output_2023_w*.csv"
)
type(df)

df.describe()
df.head()


Loaded 18 input files (4,880,579 rows)
Loaded 18 output files (562,936 rows)

✅ Merge summary:
  Input shape: (4880579, 23)
  Output shape: (562936, 6)
  Merged shape: (4880579, 25)

Crosstab (who has NaN outputs):
x_output_is_NaN     False    True 
player_to_predict                 
False                   0  3577139
True               560426   743014


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,x_output,y_output
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22,NaN,NaN
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22,NaN,NaN
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22,NaN,NaN
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22,NaN,NaN
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22,NaN,NaN


3.) Feature Engineering

In [8]:
def features(df):
        df['height_cm'] = df['player_height'].str.extract(r'(\d+)-').astype(int) * 12 + df['player_height'].str.extract(r'-(\d+)').astype(int)
        df = df.drop(columns=['player_height'])
        df =  df.rename(columns={'s': 'speed', 'a': 'acceleration', 'o': 'orientation'})
        df["player_side"] = df["player_side"].replace({"Defense": 0, "Offense": 1})
        df["play_direction"] = df["play_direction"].replace({"left": 0, "right": 1})
        return df





In [9]:
df_post = features(df)
for col in df_post.columns:
    print(col)


C:\Users\antho\AppData\Local\Temp\ipykernel_18056\3351764015.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["player_side"] = df["player_side"].replace({"Defense": 0, "Offense": 1})


game_id
play_id
player_to_predict
nfl_id
frame_id
play_direction
absolute_yardline_number
player_name
player_weight
player_birth_date
player_position
player_side
player_role
x
y
speed
acceleration
dir
orientation
num_frames_output
ball_land_x
ball_land_y
x_output
y_output
height_cm


C:\Users\antho\AppData\Local\Temp\ipykernel_18056\3351764015.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["play_direction"] = df["play_direction"].replace({"left": 0, "right": 1})


Splitting and Training

In [10]:
df_post = df_post.dropna(subset=["x_output"])
df_post = df_post.dropna(subset=["y_output"])
cols_to_drop = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'player_to_predict', 'x_output', 'y_output', 'player_name','num_frames_output', 'player_birth_date']
X = df_post.drop(columns = cols_to_drop)
y = df_post[['x_output', 'y_output']]

X.head()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train Size:", X_train.shape, "|Test Size:", X_test.shape)




Train Size: (448340, 15) |Test Size: (112086, 15)


4.) Catboost Regression

In [11]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def catboost_multioutput(X, y, test_size=0.2, random_state=42,
                         iterations=400, depth=6, learning_rate=0.05):


    #Remove rows with any NaN in targets
    mask = ~y.isna().any(axis=1)
    X, y = X.loc[mask], y.loc[mask]


    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state
    )


    cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

    models = {}
    predictions = {}

    #Train one model per target column
    for target_col in y.columns:
        train_pool = Pool(X_train, y_train[target_col], cat_features=cat_cols)
        test_pool = Pool(X_test, cat_features=cat_cols)

        model = CatBoostRegressor(
            loss_function="RMSE",
            iterations=iterations,
            depth=depth,
            learning_rate=learning_rate,
            verbose=False,
            random_seed=random_state
        )
        model.fit(train_pool)

        pred = model.predict(test_pool)
        predictions[target_col] = pred
        models[target_col] = model

    #RMSE
    y_pred_all = np.column_stack([predictions[col] for col in y.columns])
    y_test_all = y_test[y.columns].values
    overall_rmse = np.sqrt(np.sum((y_test_all - y_pred_all) ** 2) / (y_test_all.shape[0] * y_test_all.shape[1]))
    print(f"\nOverall multi-output RMSE: {overall_rmse:.4f}\n")

    # RMSEs
    for target_col in y.columns:
        rmse = np.sqrt(mean_squared_error(y_test[target_col], predictions[target_col]))
        print(f"RMSE for {target_col}: {rmse:.4f}")

    return models, predictions, (X_train, X_test, y_train, y_test)


In [12]:
catboost_multioutput(X, y, test_size=0.2, random_state=42)


Overall multi-output RMSE: 3.7660

RMSE for x_output: 3.7371
RMSE for y_output: 3.7946


({'x_output': <catboost.core.CatBoostRegressor at 0x1aa5cea8ad0>,
  'y_output': <catboost.core.CatBoostRegressor at 0x1aa5cfcf890>},
 {'x_output': array([81.94733469, 61.86188735, 77.18989502, ..., 52.27053419,
         59.22117137, 99.97471424], shape=(112086,)),
  'y_output': array([15.58138547, 35.37776168,  5.1647568 , ..., 44.44839853,
         21.61267304,  8.69244004], shape=(112086,))},
 (         play_direction  absolute_yardline_number  player_weight  \
  444400                1                        85            222   
  1469213               0                        85            205   
  3879239               1                        50            201   
  3065452               0                        11            258   
  1897454               0                        69            236   
  ...                 ...                       ...            ...   
  967846                0                        87            232   
  2261700               1                 

4.) Random Forrest

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

def rf_multioutput_pipeline(X, y, test_size=0.2, random_state=42):

    X = X.copy()
    y = y.copy()

    #Drop rows with NaN
    mask = y.notna().all(axis=1)
    X = X.loc[mask]
    y = y.loc[mask]

    # categorical columns-
    for col in X.columns:
        if X[col].dtype == "object":
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Initialize and train model
    model = RandomForestRegressor(
        n_estimators=300,
        random_state=random_state,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    #Predict on test set
    preds_array = model.predict(X_test)
    preds = pd.DataFrame(preds_array, columns=y.columns, index=y_test.index)

    # Compute  RMSE
    y_test_all = y_test.values
    y_pred_all = preds.values
    overall_rmse = np.sqrt(np.mean((y_test_all - y_pred_all) ** 2))
    print(f"\nOverall multi-output RMSE: {overall_rmse:.4f}\n")

    #  Compute RMSE
    for target_col in y.columns:
        rmse = np.sqrt(mean_squared_error(y_test[target_col], preds[target_col]))
        print(f"RMSE for {target_col}: {rmse:.4f}")

    return model, preds, (X_train, X_test, y_train, y_test)


In [30]:
rf_multioutput_pipeline(X, y, test_size=0.2, random_state=42)


Overall multi-output RMSE: 0.8356

RMSE for x_output: 0.8041
RMSE for y_output: 0.8660


(RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42),
           x_output   y_output
 239355   84.083733  19.553667
 1599147  63.743100  33.213067
 713074   71.831000  10.269500
 880813   75.706867  38.140100
 3636837  40.476433  44.433367
 ...            ...        ...
 1735363  41.384600  25.324133
 3859792  62.312633   6.331300
 4388210  57.886867  29.407167
 3431211  63.375833  21.267633
 2517589  98.560267   6.595900
 
 [112086 rows x 2 columns],
 (         play_direction  absolute_yardline_number  player_weight  \
  444400                1                        85            222   
  1469213               0                        85            205   
  3879239               1                        50            201   
  3065452               0                        11            258   
  1897454               0                        69            236   
  ...                 ...                       ...            ...   
  967846                0             

5.) XGBoost

In [13]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

def xgb_multioutput_pipeline(X, y, test_size=0.2, random_state=42):

    X = X.copy()

    # Encode categoricals
    for col in X.columns:
        if X[col].dtype == "object":
            X[col] = LabelEncoder().fit_transform(X[col].astype(str))

    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Model
    model = XGBRegressor(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        tree_method="hist",
        random_state=random_state
    )

    # Train
    model.fit(X_train, y_train)

    # Predict
    preds = model.predict(X_test)

    # RMSE per output
    rmse_x1 = np.sqrt(mean_squared_error(y_test.iloc[:, 0], preds[:, 0]))
    rmse_y1 = np.sqrt(mean_squared_error(y_test.iloc[:, 1], preds[:, 1]))

    print("RMSE x1:", rmse_x1)
    print("RMSE y1:", rmse_y1)

    return model, preds, (rmse_x1, rmse_y1)
def multioutput_rmse(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    N = y_true.shape[0]
    sq_diff = (y_true - y_pred) ** 2
    rmse = np.sqrt(np.sum(sq_diff) / (2 * N))
    return rmse

In [14]:
xgb_multioutput_pipeline(X, y, test_size=0.2, random_state=42)

RMSE x1: 3.2533202433944894
RMSE y1: 3.4038044957826754


(XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=None, num_parallel_tree=None, ...),
 array([[81.70393 , 14.758024],
        [61.581863, 33.958572],
        [75.187874,  5.765513],
        ...,
        [52.23964 , 42.36799 ],
        [62.918045, 21.935009],
        [99.90566 ,  9.162045]], shape=(112086, 2), dtype=float32),
 (